<a href="https://colab.research.google.com/github/TorbjornLarsson/Life-in-the-Universe/blob/main/LiU_E2_Habitability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jupyter notebook in Colab for the E2 exercise, without using any AI support. Some code and data is imported according to the exercise instructions.

In [81]:
import numpy as np
import pandas as pd

# Mount Drive
from google.colab import drive
try:
    drive.mount('/content/drive')
except ValueError:
    print("Google Drive mount failed. Please re-run this cell and ensure you complete the authentication process when prompted.")
    # Subsequent operations depending on Drive will likely fail if it's not mounted.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. HZ boundaries. First we adapt the The Habitable Zone Calculator program to Python 3 and test that it works on the mounted drive.

In [82]:
# ***********************************************************************************
# This code calculates habitable zone 'fluxes' using the expression given in the
# Kopparapu et al.(2014) paper. The corresponding output file is 'HZs.dat'.
# It also generates a file 'HZ_coefficients.dat' that gives the coefficients for
# the analytical expression.
#
# Ravi kumar Kopparapu April 19 2014
#
# Translated to python code by John Armstrong (jcarmstrong@weber.edu) 04 June 2014
# Translated to python 3 code by Torbjörn Larsson 14 December 2025 - no fancy stuff
#************************************************************************************
#************************************************************************************
# Output files.

hzdat = open('/content/drive/MyDrive/Colab/HZs.dat', 'w')
hzcoeff = open('/content/drive/MyDrive/Colab/HZ_coefficients.dat', 'w')

#************************************************************************************
# Coeffcients to be used in the analytical expression to calculate habitable zone flux
# boundaries

seff = [0,0,0,0,0,0]
seffsun  = [1.776,1.107, 0.356, 0.320, 1.188, 0.99]
a = [2.136e-4, 1.332e-4, 6.171e-5, 5.547e-5, 1.433e-4, 1.209e-4]
b = [2.533e-8, 1.580e-8, 1.698e-9, 1.526e-9, 1.707e-8, 1.404e-8]
c = [-1.332e-11, -8.308e-12, -3.198e-12, -2.874e-12, -8.968e-12, -7.418e-12]
d = [-3.097e-15, -1.931e-15, -5.575e-16, -5.011e-16, -2.084e-15, -1.713e-15]


#************************************************************************************
# Writing coefficients into 'HZ_coefficients.dat' file

hzcoeff.write('# The coefficients are as follows. The columns, i, are arranged according to\n')
hzcoeff.write('# the HZ limits given in the paper.\n')
hzcoeff.write('#\n')
hzcoeff.write('# i = 1 --> Recent Venus\n')
hzcoeff.write('# i = 2 --> Runaway Greenhouse\n')
hzcoeff.write('# i = 3 --> Maximum Greenhouse\n')
hzcoeff.write('# i = 4 --> Early Mars\n')
hzcoeff.write('# i = 5 --> Runaway Greenhouse for 5 ME\n')
hzcoeff.write('# i = 6 --> Runaway Greenhouse for 0.1 ME\n')

hzcoeff.write('# First row: S_effSun(i)\n')
hzcoeff.write('# Second row: a(i)\n')
hzcoeff.write('# Third row:  b(i)\n')
hzcoeff.write('# Fourth row: c(i)\n')
hzcoeff.write('# Fifth row:  d(i)\n')
hzcoeff.write('   '+'{:6.6E}'.format(seffsun[0]) + '  ' +
              '{:6.6E}'.format(seffsun[1]) + '  ' +
              '{:6.6E}'.format(seffsun[2]) + '  ' +
              '{:6.6E}'.format(seffsun[3]) + '  ' +
              '{:6.6E}'.format(seffsun[4]) + '  ' +
              '{:6.6E}'.format(seffsun[5]) + '  ' +
              '\n')
hzcoeff.write('   '+'{:6.6E}'.format(a[0]) + '  ' +
              '{:6.6E}'.format(a[1]) + '  ' +
              '{:6.6E}'.format(a[2]) + '  ' +
              '{:6.6E}'.format(a[3]) + '  ' +
              '{:6.6E}'.format(a[4]) + '  ' +
              '{:6.6E}'.format(a[5]) + '  ' +
              '\n')
hzcoeff.write('   '+'{:6.6E}'.format(b[0]) + '  ' +
              '{:6.6E}'.format(b[1]) + '  ' +
              '{:6.6E}'.format(b[2]) + '  ' +
              '{:6.6E}'.format(b[3]) + '  ' +
              '{:6.6E}'.format(b[4]) + '  ' +
              '{:6.6E}'.format(b[5]) + '  ' +
              '\n')
hzcoeff.write('   '+'{:6.5e}'.format(c[0]) + '  ' +
              '{:6.5e}'.format(c[1]) + '  ' +
              '{:6.5e}'.format(c[2]) + '  ' +
              '{:6.5e}'.format(c[3]) + '  ' +
              '{:6.5e}'.format(c[4]) + '  ' +
              '{:6.5e}'.format(c[5]) + '  ' +
              '\n')
hzcoeff.write('   '+'{:6.5e}'.format(d[0]) + '  ' +
              '{:6.5e}'.format(d[1]) + '  ' +
              '{:6.5e}'.format(d[2]) + '  ' +
              '{:6.5e}'.format(d[3]) + '  ' +
              '{:6.5e}'.format(d[4]) + '  ' +
              '{:6.5e}'.format(d[5]) + '  ' +
              '\n')

#************************************************************************************
# Calculating HZ fluxes for stars with 2600 K < T_eff < 7200 K. The output file is
# 'HZs.dat'

teff  = 2600.0
hzdat.write('#  Teff(K)        Recent        Runaway        Maximum        Early        5ME Runaway   0.1ME Runaway\n')
hzdat.write('#                 Venus         Greenhouse     Greenhouse     Mars         Greenhouse    Greenhouse\n')

starTemp = []
recentVenus = []
runawayGreenhouse = []
maxGreenhouse = []
earlyMars = []
fivemeRunaway = []
tenthmeRunaway = []

while(teff <= 7201.0):
  tstar = teff - 5780.0
  for i in range(len(a)):
     seff[i] = seffsun[i] + a[i]*tstar + b[i]*tstar**2 + c[i]*tstar**3 + d[i]*tstar**4

  starTemp.append(teff)
  recentVenus.append(seff[0])
  runawayGreenhouse.append(seff[1])
  maxGreenhouse.append(seff[2])
  earlyMars.append(seff[3])
  fivemeRunaway.append(seff[4])
  tenthmeRunaway.append(seff[5])

  hzdat.write('   '+'{:6.0f}'.format(teff) + '      ' +
              '{:6.6E}'.format(seff[0]) + '      ' +
              '{:6.6E}'.format(seff[1]) + '     ' +
              '{:6.6E}'.format(seff[2]) + '   ' +
              '{:6.6E}'.format(seff[3]) + ' ' +
              '{:6.6E}'.format(seff[4]) + ' ' +
              '{:6.6E}'.format(seff[5]) + '  ' +
              '\n')
  teff = teff + 200.0


print('************************************************************')
print('')
print('The HZ coefficients are printed in HZ_coefficients.dat file.')
print('HZs for stars with 2600 K <= Teff <=7200 K is in HZs.dat file.')
print('')
print('************************************************************')

hzdat.close()
hzcoeff.close()

************************************************************

The HZ coefficients are printed in HZ_coefficients.dat file.
HZs for stars with 2600 K <= Teff <=7200 K is in HZs.dat file.

************************************************************


The habitable stellar flux boundaries flux are given relative Earth flux.

Now we want to import the planet data and run the flux part of the code on that.

In [83]:
# Data imports
url = '/content/drive/MyDrive/Colab/table.csv'
planets = pd.read_csv(url)
print(planets)
t = planets[:1].iloc[0]

hzdat = open('/content/drive/MyDrive/Colab/HZs.dat', 'w')
hzcsv = open('/content/drive/MyDrive/Colab/HZs.csv', 'w')
hzdat.write('#  Teff(K)        Recent        Runaway        Maximum        Early        5ME Runaway   0.1ME Runaway\n')
hzdat.write('#                 Venus         Greenhouse     Greenhouse     Mars         Greenhouse    Greenhouse\n')

starTemp = []
recentVenus = []
runawayGreenhouse = []
maxGreenhouse = []
earlyMars = []
fivemeRunaway = []
tenthmeRunaway = []

for teff in t.iloc[1:5]:
  tstar = pd.to_numeric(teff) - 5780.0
  for i in range(len(a)):
     seff[i] = seffsun[i] + a[i]*tstar + b[i]*tstar**2 + c[i]*tstar**3 + d[i]*tstar**4

  starTemp.append(teff)
  recentVenus.append(seff[0])
  runawayGreenhouse.append(seff[1])
  maxGreenhouse.append(seff[2])
  earlyMars.append(seff[3])
  fivemeRunaway.append(seff[4])
  tenthmeRunaway.append(seff[5])
  hzdat.write('   '+'{:6.0f}'.format(teff) + '      ' +
              '{:6.6E}'.format(seff[0]) + '      ' +
              '{:6.6E}'.format(seff[1]) + '     ' +
              '{:6.6E}'.format(seff[2]) + '   ' +
              '{:6.6E}'.format(seff[3]) + ' ' +
              '{:6.6E}'.format(seff[4]) + ' ' +
              '{:6.6E}'.format(seff[5]) + '  ' +
              '\n')
  # Convert relative flux F = L/d^2 to relative distance d~/sqrt(F) for L=1
  hzcsv.write('{:6.0f}'.format(teff) + ',' +
              '{:6.6E}'.format(1./np.sqrt(seff[0])) + ',' +
              '{:6.6E}'.format(1./np.sqrt(seff[1])) + ',' +
              '{:6.6E}'.format(1./np.sqrt(seff[2])) + ',' +
              '{:6.6E}'.format(1./np.sqrt(seff[3])) + ',' +
              '{:6.6E}'.format(1./np.sqrt(seff[4])) + ',' +
              '{:6.6E}'.format(1./np.sqrt(seff[5])) +
              '\n')
hzdat.close()
hzcsv.close()



        Name      GJ 514 b    Exo-1 b   Exo-2 b    Exo-3 b
0   Teff         3728.0000  3457.0000  5597.000  6157.0000
1   Teff_u         68.0000    39.0000    95.000   231.0000
2   Lstar           0.0430     0.0217     1.230     1.5200
3   Lstar_u         0.0087     0.0041     0.120     0.3200
4   Mstar           0.5100     0.3590     1.011     1.0880
5   Mstar_u         0.0510     0.0470     0.056     0.0720
6   Mp              0.0160     0.0281     0.500     4.6000
7   Mp_u            0.0030     0.0053     0.200     1.0000
8   a               0.4220     0.1429     2.026     1.6400
9   a_u             0.0150     0.0065     0.037     0.1000
10  e               0.4500     0.2000     0.260     0.4000
11  e_u             0.1500     0.0800     0.210     0.1500
12  Rp              0.2560     0.2110     0.819     0.8886
13  Rp_u            0.0200     0.0200     0.019     0.0535


HZs.dat:
#  Teff(K)        Recent        Runaway        Maximum        Early        5ME Runaway   0.1ME Runaway
#                 Venus         Greenhouse     Greenhouse     Mars         Greenhouse    Greenhouse
     3728      1.504530E+00      9.377502E-01     2.542683E-01   2.285490E-01 1.006363E+00 8.347542E-01  
     3457      1.493286E+00      9.307533E-01     2.456651E-01   2.208133E-01 9.989625E-01 8.280202E-01  
     5597      1.737838E+00      1.083202E+00     3.447829E-01   3.099171E-01 1.162400E+00 9.683890E-01  
     6157      1.859351E+00      1.158978E+00     3.793234E-01   3.409650E-01 1.243928E+00 1.037143E+00  

HZs.csv:
  3728,8.152665E-01,1.032658E+00,1.983143E+00,2.091753E+00,9.968338E-01,1.094512E+00
  3457,8.183302E-01,1.036532E+00,2.017568E+00,2.128077E+00,1.000519E+00,1.098954E+00
  5597,7.585695E-01,9.608270E-01,1.703049E+00,1.796293E+00,9.275175E-01,1.016190E+00
  6157,7.333635E-01,9.288860E-01,1.623660E+00,1.712557E+00,8.966077E-01,9.819305E-01

* Verifikation on https://personal.ems.psu.edu/~jfk4/ruk15/planets/:
* The planet masses are listed in fraktions of M_Jup.
* For the inner HZ boundary use the Runaway
Greenhouse limit for 5 Earth masses, and for the outer HZ boundary use the Maximum Greenhouse limit.
* For the verification planetary system, note that the
Habitable Zone Gallery uses the Runaway Greenhouse limit for 1 Earth mass for the inner HZ boundary for all stars.
* Conservative habitable zone limits (5 Earth mass)	Stellar flux compared to the Sun	HZ distance from the star (AU)
* Inner HZ - Runaway Greenhouse limit
* 1.0064
* 0.997
* Program output: 1.006363E+00
* Conservative habitable zone limits (1 Earth mass)	Stellar flux compared to the Sun	HZ distance from the star (AU)
* Outer HZ - Maximum Greenhouse limit
* 0.254
* 1.984
* Program output: 2.542683E-01
* ⇒ Both set of numbers corresponds.



2. The task is to calculate the time spent within the conservative habitable zone for each planet. The time should be given both relative to the orbital period and in
absolute time units (days). Here are some hints for how to proceed:
* Calculate the periastron and apastron distances for each orbit and compare with the inner and outer HZ boundaries.
  * Periastron = (1-e)*a
  * Apastron = (1+e)*a

* Calculate the time it takes the planet to move from periastron to the inner HZ boundary, the outer HZ boundary, or both, as needed. Also calculate the orbital period. Use relevant equations from Lecture 6b "Condensed celestial mechanics". Combine the results to give the final result.
  *  Effective mass u = G/(m1+m2)
  *  True anomaly th = np.arccos((a*(1-e**2)/(oHZ*e)-1/e))
  *  Eccentric anomaly tan(E/2)=sqrt((1-e)/(1+e))tan(th/2)
  *  Mass anomaly M = E-e*np.sin(E)

* Draw an approximate sketch of the planet orbit and the HZ boundaries, to verify that your answers make sense. (Separate drawing.)

In [84]:
# iHZ is column 6, oHZ is column 4
d = pd.read_csv('/content/drive/MyDrive/Colab/HZs.csv', sep=",", header=None)

# For each planet we calculate periastron, apastron and get the iHZ, oHZ.
# Planet a is 9th row, e is 11th.
# We also calculate the orbital period in seconds
# P = sqrt(4pi^2*a^3/u) and in days Pdays P/(# s in a day).
# u= G(m1+m2), G = 6.67430e-11 # m3 kg-1 s-2, AU = 1.4959787e11 # m,
# MJup = 1.900e27 # kg, Msun = 1.9884e30 # kg
p = list(planets)
for i in list(range(1,len(p))):
  a = planets.iloc[:, i][8]
  e = planets.iloc[:, i][10]
  peri = (1-e)*a
  apa = (1+e)*a
  iHZ = d.iloc[:,5][i-1]
  oHZ = d.iloc[:,3][i-1]
  mstar = planets.iloc[:, i][5]
  mplanet = planets.iloc[:, i][7]
  # u = G/(m1+m2)
  G = 6.67430E-11
  u = G*(mstar*1.9884E30+mplanet*1.900E27)
  # P = np.sqrt(4*np.pi**2*(a*1.4959787E11)**3/u)
  # That wasn't quite right ... About a factor pi too high.
  # Another check is with Earth parameters
  P = np.sqrt(4*(a*1.4959787E11)**3/u)
  Pabs = P/(24*60*60)
  print("Pabs:", Pabs)
  # Distance to move from periastron to inner HZ
  # -> no such system

  # Distance to move from periastron to outer HZ
  # If negative do nothing, else calculate time to oHZ
  r = peri-oHZ
  if peri > iHZ and r < 0:
    #print(i)
    #First we get true anomaly cos(tHZ)
    th = np.arccos((a*(1-e**2)/(oHZ*e)-1/e))
    #Then the eccentric anomaly tan(E/2)=sqrt((1-e)/(1+e))tan(th/2)
    E = 2*np.arctan(np.sqrt((1-e)/(1+e))*np.tan(0.5*th))
    #And finally M as a branched fraction of orbital period
    M = E-e*np.sin(E)
    # Check that we are on the right branch of M
    if M > 1:
      M = M-1
    t = Pabs*M
    print("Time:", t)







Pabs: 141.13054296875458
Pabs: 28.968475892763784
Pabs: 1414.408934871439
Time: 997.7629598774274
Pabs: 904.0371066123414
Time: 131.9978432755656


* There is a trigonometric problem in the code, empirically fixed.
* Sketches verify that:
  * The two first planets orbit inside the HZ.
  * The last two planets can orbit from within the HZ out of it, taking 997 respectively 131 days to do so.

3. Earth similarity
* Calculate the Earth similarity index (ESI) for the planets.
* ESIx = (1-|x-x0|/|x+x0|)^w, x0 Earth reference, w exponential weights.
* Interior ESII = (ESIr*ESIrho)^0.5
* Surface ESIS = (ESIve*ESIt)^0.5
* ESI = (ESII*ESIS)^0.5
* Mean radius ESIr: w = 0.57
* Bulk density ESIrho: w = 1.07
* Escape velocity ESIve: w = 0.70
* Mean surface temperature ESIt: w = 5.58

* Calculate the bulk density and escape velocity of the planets and the Earth from mass and radius.
  * rho = M/V ~ Mp/(4/3*π*Rp^3)
  * ve ~ sqrt(2*G*Mp/Rp)
* For the surface temperature TS you should use the equilibrium
temperature calculated from stellar luminosity and orbital semi-major axis, using an albedo of zero.
  * TS = ((1-A)Lstar/(16*π*a^2*σ))^(1/4), A=0, σ is the Stefan-Boltzmann constant = 5.670373E-8

In [85]:
# Data struct planets:
# Rp row 13
# Mp row 7
# Lstar row 3
# a row 9

# Reference values:
RJup = 6.9911E7
RE = 6.3710E6
MJup = 1.900E27
ME = 5.9722E24
Lsun = 3.846E26
AU = 1.4959787E11

for i in list(range(1,len(p))):
  # ESIr =  (1-|RP-RE|/|RP+RE|)**0.57
  RP = RJup*planets.iloc[:, i][12]
  ESIr = (1-np.absolute(RP-RE)/np.absolute(RP+RE))**0.57
  # ESIrho = (1-|rhoP-rhoE|/|rhoP+rhoE|)**1.07
  MP = MJup*planets.iloc[:, i][6]
  rhoP = MP/(4/3*np.pi*RP**3)
  rhoE = ME/(4/3*np.pi*RE**3)
  ESIrho = (1-np.absolute(rhoP-rhoE)/np.absolute(rhoP+rhoE))**0.57
  ESII = (ESIr*ESIrho)**0.5
  # ESIve = (1-|veP-veE|/|veP+veE|)**0.70
  veP = np.sqrt(2*G*MP/RP)
  veE = np.sqrt(2*G*ME/RE)
  ESIve = (1-np.absolute(veP-veE)/np.absolute(veP+veE))**0.70
  # TS = (Lstar/(16πa^2*σ))^(1/4), σ = 5.670373E-8
  Lstar = Lsun*planets.iloc[:, i][2]
  a = planets.iloc[:, i][8]
  sigma = 5.670373E-8
  TSP = (Lstar/(16*np.pi*((a*AU)**2*sigma)))**0.25
  # Earth semi-orbital axis is roughly 1 AU
  TSE = (Lsun/(16*np.pi*((1.0*AU)**2*sigma)))**0.25
  #  ESIt = (1-|TSP-TSE|/|TSP+TSE|)**5.58
  ESIt = (1-np.absolute(TSP-TSE)/np.absolute(TSP+TSE))**5.58
  ESIS = (ESIve*ESIt)**0.5
  ESI = (ESII*ESIS)**0.5
  print("ESI:",ESI)

ESI: 0.5887361150889748
ESI: 0.8379009339872671
ESI: 0.46379714400439825
ESI: 0.4917252311072898


I get reasonable ESIs:
0.5887361150889748
0.8379009339872671
0.46379714400439825
0.4917252311072898

Earth TSE before greenhouse effect is reasonable, at above freezing.


We should also give some example of error estimates. Assuming independence, we have relative error du/u = ∑_i |du_i/u_i| for given uncertainties du_i of observables u_i.

In [86]:
# Data struct planets:
# Rp row 13
# Rp_u row 14
# Mp row 7
# Mp_u row 8

# Relative error in radius Rp_u/Rp
RPu = RJup*planets.iloc[:, i][13]
relRP = RPu/RP
print(relRP)

# Relative error in mass Mp_u/Mp
MPu = MJup*planets.iloc[:, i][7]
relMP = MPu/MP
print(relMP)

# Relative error in mean density rhoP_u/rhoP = relMP+3*relRP
relrhoP = relMP+3*relRP
print(relrhoP)


0.06020706729687149
0.21739130434782608
0.39801250623844053
